In [1]:
import requests
from datetime import datetime
import traceback
import time
import json
import logging
from tqdm.notebook import tqdm
import pandas as pd

## Set up Logging

In [2]:
logger = logging.getLogger("my app")
logger.setLevel("DEBUG")
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

## Global Variables

In [3]:
subreddit = "dataengineering"

In [4]:
base_url = "https://api.pushshift.io/reddit/submission/search?\
&subreddit={subreddit}\
&limit={limit}\
&sort={sort}\
&sort_type={sort_type}\
&before={end_timestamp}\
&{start_timestamp}"

In [5]:
valid_columns = [
    'author',
    'author_flair_css_class',
    'author_flair_text',
    'author_premium',
    'created_utc',
    'domain',
    'full_link',
    'id',
    'is_robot_indexable',
    'is_self',
    'is_video',
    'link_flair_text',
    'no_follow',
    'num_comments',
    'permalink',
    'post_hint',
    'score',
    'selftext',
    'title',
    'url',
    'url_overridden_by_dest',
    'removed_by_category',
    'banned_by',
    'removed_by',
]

## helper functions

In [6]:
def _get_post(post):
    filtered_posts = {k:v for (k,v) in post.items() if k in valid_columns}
    return filtered_posts
                     
def get_posts(json_objects):
    posts = []
    for post in tqdm(json_objects, len(posts)):
        post_args = _get_post(post)
        posts.append(post_args)
    return posts

## Get all subreddit posts

In [7]:
def get_all_subreddit_posts(pushshift_args, full_refresh = True):
    all_posts = []
    start_time = datetime.utcnow()
    previous_epoch = int(start_time.timestamp())
    if not full_refresh:
        pushshift_args["start_timestamp"] = None
    while True: # loop through requests until empty
        pushshift_args["end_timestamp"] = previous_epoch
        request_url = base_url.format(**pushshift_args)
        json_text = requests.get(request_url) # get reddit data via request from pushshift
        time.sleep(1)
        try: # parse request to json
            json_request = json_text.json()
        except json.decoder.JSONDecodeError as e:
            logger.warning("Unable to Decode json request:", e)
            time.sleep(1)
            continue
            
        json_objects = json_request.get('data', [])
        if not json_objects: # check if result set is empty
            logger.warning("Empty json request")
            break
            
        previous_epoch = json_objects[-1]['created_utc'] - 1
        total_posts = len(json_objects)
            
        posts = get_posts(json_objects) # get posts from request
        all_posts.extend(posts)
        logger.info(f"Processed {len(all_posts)} posts")

    logger.info(f"Saved {len(all_posts)} posts")
    return all_posts

In [8]:
pushshift_args = {
    'subreddit': 'dataengineering',
    'limit': 1000,
    'sort': 'desc',
    'sort_type': 'created_utc',
    'end_timestamp': None,
    'start_timestamp': None
}

In [9]:
all_posts = get_all_subreddit_posts(pushshift_args)

  0%|          | 0/100 [00:00<?, ?it/s]

2021-12-26 14:48:09,186 - my app - INFO - Processed 100 posts


  0%|          | 0/100 [00:00<?, ?it/s]

2021-12-26 14:48:11,098 - my app - INFO - Processed 200 posts


  0%|          | 0/100 [00:00<?, ?it/s]

...

2021-12-26 14:50:58,746 - my app - INFO - Processed 8996 posts


  0%|          | 0/100 [00:00<?, ?it/s]

2021-12-26 14:51:00,349 - my app - INFO - Processed 9096 posts


  0%|          | 0/10 [00:00<?, ?it/s]

2021-12-26 14:51:02,011 - my app - INFO - Processed 9106 posts
2021-12-26 14:51:03,222 - my app - WARNING - Empty json request
2021-12-26 14:51:03,224 - my app - INFO - Saved 9106 posts


### Create dataframe from results

In [10]:
df = pd.DataFrame(all_posts)

### Drop Duplicates

In [11]:
df_deduped = df.drop_duplicates(['author', 'title', 'created_utc'])

### Save Dataframe to CSV

In [12]:
df_deduped.to_csv("reddit_de_posts_filtered.csv", index = False)

## Save results to Postgres Database

### Connect to Postgres Database

In [13]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv("./postgres/.env")

True

In [14]:
PG_USER = os.getenv('POSTGRES_USER')
PG_PW = os.getenv('POSTGRES_PASSWORD')
PG_DB = os.getenv('POSTGRES_DB')

In [15]:
postgres_conn_args = {
    "host": "localhost",
    "database": PG_DB,
    "user": PG_USER,
    "password": PG_PW,
    "port": 5432
}

In [16]:
pg_conn = psycopg2.connect(**postgres_conn_args)

### Create table

In [17]:
CREATE_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS raw_data.reddit_de (
    id char(6) NOT NULL,
    author text NOT NULL,
    author_flair_css_class text,
    author_flair_text text,
    author_premium text,
    created_utc bigint NOT NULL,
    domain text,
    full_link text NOT NULL,
    is_robot_indexable bool,
    is_self bool,
    is_video bool,
    link_flair_text text,
    no_follow text,
    num_comments int,
    permalink text NOT NULL,
    post_hint text,
    score int,
    selftext text,
    title text NOT NULL,
    url text NOT NULL,
    url_overridden_by_dest text,
    removed_by_category text,
    banned_by text,
    removed_by text
);
"""

In [18]:
with pg_conn:
    with pg_conn.cursor() as pg_cur:
            pg_cur.execute(CREATE_TABLE_SQL)

### Copy Contents to table

In [19]:
COPY_SQL = """
COPY raw_data.reddit_de {columns}
FROM STDIN
WITH CSV HEADER
DELIMITER as ','
"""

In [20]:
column_names = f"({', '.join(list(df_deduped.columns))})"

In [21]:
with pg_conn:
    with pg_conn.cursor() as pg_cur:
        with open("reddit_de_posts_filtered.csv",  'rb') as tmp_file:
            pg_cur.copy_expert(COPY_SQL.format(columns=column_names), tmp_file)

### Clean Up

In [22]:
with pg_conn:
    with pg_conn.cursor() as pg_cur:
        pg_cur.execute("ANALYZE")

In [23]:
pg_conn.close()

In [24]:
!rm -f 'reddit_de_posts_filtered.csv'

In [25]:
with pg_conn:
    df = pd.read_sql("SELECT * FROM raw_data.reddit_de", pg_conn)

InterfaceError: connection already closed

In [ ]:
df.shape

In [ ]:
df.id.nunique()

In [ ]:
df.post_hint.value_counts()

In [ ]:
df.columns

In [ ]:
test = df[~df.url_overridden_by_dest.isnull()]

In [ ]:
test.full_link.equals(test.url_overridden_by_dest)

In [ ]:
df.full_link.str[22:].equals(df.permalink)

In [ ]:
df.permalink.iloc[0]

In [ ]:
df.iloc[0].full_link

In [ ]:
df.iloc[0].permalink

In [ ]:
df.no_follow.value_counts()

In [ ]:
import numpy as np

In [ ]:
np.where(df[df.is_self].selftext.str.len() == 1)

In [ ]:
df[df.is_self].selftext.iloc[231]

In [ ]:
df[df.title.str.len() < 5].title

In [ ]:
df.iloc[2397]

In [ ]:
df.is_robot_indexable.value_counts()

In [ ]:
pd.DataFrame(df.author.value_counts())

In [ ]:
df[df.author == "[deleted]"]